# Import liberaries

In [1]:
import pip

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
import_or_install("qdrant_client")
import_or_install("PyPDF2")
import_or_install("sentence_transformers")

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Modules

In [37]:
%reload_ext autoreload
%autoreload 2
from Helper_Modules.PdfReader import pdf_to_docVec
from Helper_Modules.Qdrant import Qdrant

# Encoding documents and paragraphs

In [15]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [38]:
documents = [pdf_to_docVec('/content/drive/MyDrive/Memory_machine/Forschung.pdf', encoder),
             pdf_to_docVec('/content/drive/MyDrive/Memory_machine/Pflichtpraktikum.pdf', encoder),
             pdf_to_docVec('/content/drive/MyDrive/Memory_machine/ZulassungVorrausetzung_master.pdf', encoder)]

# Creating a qdrant client

In [42]:
client = Qdrant(url="",
    api_key="", encoder=encoder)

# Adding vectors
- A collection for the user name will be automatically created, if it doesn't exist.

- Otherwise the vectors will be added to the existing collection

In [43]:
userName = "testUser"

for document in documents:
  client.add_docVec(userName, document)

user doesn't exist, creating a new collection with the user name: testUser
Die Forschung an deutschen Universitäten nimmt eine herausragende Stellung in der 
internationalen Wissenschaftslandschaft ein. Deutschland ist bekannt für seine exzellente 
Forschung und seine zahlreichen renommierten Universitäten, die eine breite Palette von 
Disziplinen abdecken. Hier sind einige wichtige Aspekte der Forschung an deutschen 
Universitäten : 
 
Forschungslandschaft : 
Deutschland verfügt über eine vielfältige Forschungslandschaft mit einer beeindruckenden 
Bandbreite an Forschungsbereichen und Disziplinen. Dies umfasst Naturwissenschaften, 
Ingenieurwissenschaften, Geistes - und Sozialwissenschaften, Medizin und viele weit ere 
Fachgebiete 
 
Forschungsförderung : 
Die deutsche Regierung und verschiedene Organisationen investieren erhebliche Mittel in die 
Forschung. Das Bundesministerium für Bildung und Forschung (BMBF) sowie die Deutsche 
Forschungsgemeinschaft (DFG) spielen eine zentrale Ro

# search

In [45]:
client.search("test","Invest in der Forschung")

{'isDoc': True, 'name': 'Forschung.pdf'} score: 0.33223948
The Vector with the highst score: Forschung.pdf
{'isDoc': False, 'name': ' \nForschungsförderung : \nDie deutsche Regierung und verschiedene Organisationen investieren erhebliche Mittel in die \nForschung. Das Bundesministerium für Bildung und Forschung (BMBF) sowie die Deutsche \nForschungsgemeinschaft (DFG) spielen eine zentrale Rolle bei der Bereitstellung  von \nFinanzmitteln für die Forschung ', 'source_doc': 'Forschung.pdf'} score: 0.5662128
{'isDoc': False, 'name': 'Dies ermöglicht den Wissenschaftlern den Zugang zu einem globalen Netzwerk von \nForschungspartnern und Ressourcen ', 'source_doc': 'Forschung.pdf'} score: 0.39979255
{'isDoc': False, 'name': ' \nForschungsfelder : \nDeutschland ist in vielen Bereichen führend, darunter Maschinenbau, Chemie, Informatik, \nMedizin und erneuerbare Energien. Die Forschung in diesen Feldern trägt nicht nur zur \nwissenschaftlichen Erkenntnis bei, sondern hat auch erhebliche wirts